In [1]:
import cvxpy as cp
import numpy as np
import torch
from cvxpylayers.torch import CvxpyLayer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [2]:
np.random.seed(0)
m = 1000
n = 10
k = 2
A = np.random.rand(m, k).dot(np.random.rand(k, n))
tmp = np.random.choice(range(m*n), int(0.2*m*n),replace=False)
Anew = A.copy()
Anew[tmp//n, tmp % n] = np.nan

In [3]:
missing_list = np.transpose(np.argwhere(np.isnan(Anew)))

In [4]:
mask = np.ones((m,n))
mask[missing_list[0], missing_list[1]] = 0.0

In [5]:
Y_init = np.random.rand(m, k)
Y = torch.from_numpy(Y_init).cpu()

MAX_ITERS = 50
residual = np.zeros(MAX_ITERS)

In [6]:
%%time
for iter_num in range(1, 1+MAX_ITERS):
    if iter_num % 2 == 1:
        Xsol = cp.Variable(shape=(k, n))
        constraint = [Xsol >= 0]
        Ysol = cp.Parameter(shape=(m,k))
        Ysol.value = Y.cpu().numpy()
        obj = cp.Minimize(cp.norm( cp.multiply(mask, Anew - Ysol*Xsol), 'fro'))
        prob = cp.Problem(obj, constraint)        
        cvxpylayer = CvxpyLayer(prob, parameters=[Ysol], variables=[Xsol])
        X, = cvxpylayer(Y)
    else:
        Ysol = cp.Variable(shape=(m, k))
        constraint = [Ysol >= 0]
        Xsol = cp.Parameter(shape=(k,n))
        Xsol.value = X.cpu().numpy()
        obj = cp.Minimize(cp.norm( cp.multiply(mask, Anew - Ysol*Xsol), 'fro'))
        prob = cp.Problem(obj, constraint) 
        cvxpylayer = CvxpyLayer(prob, parameters=[Xsol], variables=[Ysol])
        Y, = cvxpylayer(X)
    rmse = np.sqrt(np.mean((np.dot(Y.cpu().numpy(), X.cpu().numpy()) -A)**2))
    print('Iteration {}, residual norm {}'.format(iter_num, rmse))
    residual[iter_num-1] = rmse

Iteration 1, residual norm 0.2545643155505715
Iteration 2, residual norm 0.07781867001255025
Iteration 3, residual norm 0.06581232683592199
Iteration 4, residual norm 0.0379420126700111
Iteration 5, residual norm 0.02766304573871009
Iteration 6, residual norm 0.022335675175930807
Iteration 7, residual norm 0.017833099418742743
Iteration 8, residual norm 0.014892972491910003
Iteration 9, residual norm 0.012463756874989162
Iteration 10, residual norm 0.010712462838441244
Iteration 11, residual norm 0.009288183051974467
Iteration 12, residual norm 0.008193859731171234
Iteration 13, residual norm 0.0072865915844929
Iteration 14, residual norm 0.006548948375814247
Iteration 15, residual norm 0.005921811470202041
Iteration 16, residual norm 0.005394115945272744
Iteration 17, residual norm 0.00493680678899824
Iteration 18, residual norm 0.004540843311986531
Iteration 19, residual norm 0.004195755054699086
Iteration 20, residual norm 0.0038884164044674877
Iteration 21, residual norm 0.00361949

In [7]:
import matplotlib.pyplot as plt
plt.plot(np.log(residual))
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
https://www.cvxpy.org/tutorial/advanced/index.html
    https://www.cvxpy.org/examples/applications/nonneg_matrix_fact.html

SyntaxError: invalid syntax (<ipython-input-8-19191d4b0c60>, line 1)

In [ ]:
%%time
Y_init = np.random.rand(m, k)
Y = Y_init
for iter_num in range(1, 1+MAX_ITERS):
    if iter_num % 2 == 1:
        X = cp.Variable(shape=(k, n))
        constraint = [X >= 0]
        obj = cp.Minimize(cp.norm( cp.multiply(mask, Anew - Y*X), 'fro'))
        prob = cp.Problem(obj, constraint)
        prob.solve(solver=cp.SCS, max_iters=10000)
        X = X.value
    else:
        Y = cp.Variable(shape=(m, k))
        constraint = [Y >= 0]
        obj = cp.Minimize(cp.norm( cp.multiply(mask, Anew - Y*X), 'fro'))
        prob = cp.Problem(obj, constraint)
        prob.solve(solver=cp.SCS, max_iters=10000)
        Y = Y.value
    rmse = np.sqrt(np.mean((A-np.dot(Y,X))**2))
    print('Iteration {}, residual norm {}'.format(iter_num, rmse))
    residual[iter_num-1] = rmse

In [ ]:
np.sqrt(np.mean((A-np.dot(Y,X))**2))


In [ ]:
X.shape